In [12]:
from joblib import load
import os

# Load the models
model0 = load(os.path.join('trials\\binary_classification_log_mel_energy\\class_0_SVM\\model.joblib'))
model1 = load(os.path.join('trials\\binary_classification_log_mel_energy\\class_1_SVM\\model.joblib'))
model2 = load(os.path.join('trials\\binary_classification_log_mel_energy\\class_2_SVM\\model.joblib'))
model3 = load(os.path.join('trials\\binary_classification_log_mel_energy\\class_3_SVM\\model.joblib'))
model_gender = load(os.path.join('trials\\log_mel_energy_Gender_Age\\Gender\\model.joblib'))

print("Models loaded successfully.")

Models loaded successfully.


d:\ProgramFiles\Python312\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
X_train = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\X_train.joblib'))
X_train2 = load(os.path.join('trials\\features\\48k_log_mel_energy_all_data\\X_train.joblib'))
y_train = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\y_train.joblib'))

X_test = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\X_test.joblib'))
X_test2 = load(os.path.join('trials\\features\\48k_log_mel_energy_all_data\\X_test.joblib'))
y_test = load(os.path.join('trials\\features\\48k_mfcc_extra_hfcc_extra\\y_test.joblib'))

In [16]:
import numpy as np
X_train_combined = np.concatenate((X_train[:,:150], X_train[:,7*75:9*75], X_train2[:,-75:]), axis=1)
X_test_combined = np.concatenate((X_test[:,:150], X_test[:,7*75:9*75], X_test2[:,-75:]), axis=1)
print(X_train_combined.shape)
print(X_test_combined.shape)

(172786, 375)
(9094, 375)


In [19]:
from sklearn.base import BaseEstimator, ClassifierMixin
import numpy as np
import time
from tqdm import tqdm

class CombinedModel(BaseEstimator, ClassifierMixin):
    def __init__(self, model0=None, model1=None, model2=None, model3=None, model_gender=None):
        self.model0 = model0
        self.model1 = model1
        self.model2 = model2
        self.model3 = model3
        self.model_gender = model_gender
        

    def predict_one(self, X):
        # Collect predictions from all models
        

        # if prediction3 == 0:
        #     prediction1 = self.model1.predict(X)
        #     if prediction1 == 0:
        #         prediction2 = self.model2.predict(X)
        #         if prediction2 == 0:
        #             final_predictions = 0
        #         else:
        #             prediction0 = self.model0.predict(X)
        #             if prediction0 == 0:
        #                 final_predictions = 2
        #             else:
        #                 final_predictions = 0
        #     else:
        #         prediction2 = self.model2.predict(X)
        #         if prediction2 == 0:
        #             final_predictions = 1
        #         else:
        #             final_predictions = 2
        # else:
        #     prediction1 = self.model1.predict(X)
        #     if prediction1 == 0:
        #         prediction2 = self.model2.predict(X)
        #         if prediction2 == 0:
        #             final_predictions = 3
        #         else:
        #             final_predictions = 2
        #     else:
        #         prediction0 = self.model0.predict(X)
        #         if prediction0 == 0:
        #             final_predictions = 3
        #         else:
        #             final_predictions = 1

        final_predictions = None
        prediction_gender = model_gender.predict(X)
        if prediction_gender == 0:
            prediction0 = self.model0.predict(X)
            prediction2 = self.model2.predict(X)

            if prediction0 == 1:
                final_predictions = 0
            elif prediction2 == 1:
                final_predictions = 2
            else:
                prediction1 = self.model1.predict(X)
                if prediction1 == 1:
                    final_predictions = 1
                else:
                    final_predictions = 0
            
        else:
            prediction1 = self.model1.predict(X)
            prediction3 = self.model3.predict(X)
            if prediction1 == 1:
                final_predictions = 1
            elif prediction3 == 1:
                final_predictions = 3
            else:
                prediction0 = self.model0.predict(X)
                if prediction0 == 1:
                    final_predictions = 3
                else:
                    final_predictions = 2

        return final_predictions

        
        
    def predict(self, X):
        # Apply predict_one for each sample in X
        return np.array([self.predict_one(sample.reshape(1, -1)) for sample in tqdm(X, desc="Predicting")])
# Create an instance of the combined model
combined_model = CombinedModel(model0=model0, model1=model1, model2=model2, model3=model3, model_gender=model_gender)
# Measure the time taken for one prediction
start_time = time.time()
combined_model.predict(X_val_fold[:1])  # Predict for one sample
end_time = time.time()

print(f"Time taken for one prediction: {end_time - start_time:.6f} seconds")


Predicting: 100%|██████████| 1/1 [00:00<00:00, 32.26it/s]

Time taken for one prediction: 0.033996 seconds


In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


skf = StratifiedKFold(n_splits=20, shuffle=True, random_state=42)
train_idx, val_idx = next(skf.split(X_train_combined, y_train))

X_train_fold, X_val_fold = X_train_combined[train_idx], X_train_combined[val_idx]
y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]
X_train_balanced, y_train_balanced = X_train_fold, y_train_fold


y_val_pred = combined_model.predict(X_val_fold)
#y_val_pred_proba_3 = clf.predict_proba(X_val_fold)


Predicting: 100%|██████████| 8640/8640 [02:41<00:00, 53.66it/s]


In [23]:
from sklearn.metrics import  balanced_accuracy_score

print('Balanced Accuracy: %.9f' % (balanced_accuracy_score(y_val_fold, y_val_pred)))
print('Accuracy: %.9f' % (accuracy_score(y_val_fold, y_val_pred)))
print(classification_report(y_val_fold, y_val_pred))
print(confusion_matrix(y_val_fold, y_val_pred))

Balanced Accuracy: 0.997539305
Accuracy: 0.998958333
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5907
           1       1.00      1.00      1.00      1029
           2       1.00      0.99      1.00       922
           3       1.00      1.00      1.00       782

    accuracy                           1.00      8640
   macro avg       1.00      1.00      1.00      8640
weighted avg       1.00      1.00      1.00      8640

[[5907    0    0    0]
 [   0 1028    1    0]
 [   7    0  915    0]
 [   0    0    1  781]]


In [21]:
#Predict on the test set
y_test_pred = combined_model.predict(X_test_combined)

# Print accuracy
print('Test Accuracy: %.9f' % (accuracy_score(y_test, y_test_pred)))

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_test_pred))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

Predicting: 100%|██████████| 9094/9094 [02:51<00:00, 53.01it/s]

Test Accuracy: 0.974598636
Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      6217
           1       0.98      0.96      0.97      1084
           2       0.93      0.90      0.91       971
           3       0.97      0.95      0.96       822

    accuracy                           0.97      9094
   macro avg       0.97      0.95      0.96      9094
weighted avg       0.97      0.97      0.97      9094

Confusion Matrix:
[[6167    8   26   16]
 [  12 1042   23    7]
 [  98    0  873    0]
 [  14   11   16  781]]


In [27]:
print('Balanced Accuracy: %.9f' % (balanced_accuracy_score(y_test, y_test_pred)))


Balanced Accuracy: 0.950601731
